# Обучение сиамской сети для различия двух объектов

In [6]:
import cv2
import matplotlib as plt
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch import Generator
from torch.utils.data import ConcatDataset, DataLoader, random_split

from src.config import MOT20_EXT_FIRST_AXIS_MEAN, MOT20_EXT_SECOND_AXIS_MEAN
from src.data import MOT20ExtDataset
from src.train import display_images, get_statistics
from src.transforms import get_norm_transform, get_resize_transform

## Загрузка данных

In [9]:
resize_transform = get_resize_transform((MOT20_EXT_FIRST_AXIS_MEAN, MOT20_EXT_SECOND_AXIS_MEAN)) 

In [11]:
dataset01 = MOT20ExtDataset('data/MOT20_ext/train/MOT20-01/', transform=resize_transform)
dataset02 = MOT20ExtDataset('data/MOT20_ext/train/MOT20-02/', transform=resize_transform)
dataset03 = MOT20ExtDataset('data/MOT20_ext/train/MOT20-03/', transform=resize_transform)
dataset05 = MOT20ExtDataset('data/MOT20_ext/train/MOT20-05/', transform=resize_transform)

In [12]:
dataset = ConcatDataset([dataset01, dataset02, dataset03, dataset05])

In [13]:
len(dataset)

135721

### Создание даталоадеров

In [14]:
TEST_PROPROTION = 0.2
VAL_PROPORTION = 0.15
TRAIN_PROPORTION = 1 - TEST_PROPROTION - VAL_PROPORTION
sum([TEST_PROPROTION, VAL_PROPORTION, TRAIN_PROPORTION])

1.0

In [15]:
generator = torch.manual_seed(0)

In [16]:
train_set, val_set, test_set = random_split(
    dataset, [TRAIN_PROPORTION, VAL_PROPORTION, TEST_PROPROTION], generator=generator)

In [17]:
BATCH_SIZE = 64

In [18]:
train_loader = DataLoader(
    train_set,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
    generator=generator,
    
)

val_loader = DataLoader(
    val_set,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
    generator=generator
)

test_loader = DataLoader(
    test_set,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
    generator=generator
)

In [19]:
len(train_loader), len(val_loader), len(test_loader)

(1378, 318, 424)

### Расчет статистик

_Примечание для себя: лоадер создает итератор, который возвращает тапл: словарь, словарь, тензор с метками. Словарь содержит тензор из BATCH_SIZE картинок по ключу image_

In [20]:
loader = DataLoader(
    dataset,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
    generator=generator,
)

In [24]:
get_statistics(loader)

  0%|          | 0/2120 [00:53<?, ?it/s]


AttributeError: 'list' object has no attribute 'size'

### Аугментация и нормализация